In [54]:
pip install together PyMuPDF tiktoken pandas


In [56]:
import fitz  # PyMuPDF
import tiktoken

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

def chunk_text(text, chunk_size=1000):
    """Splits text into manageable chunks based on token limits."""
    tokenizer = tiktoken.get_encoding("cl100k_base")  # OpenAI tokenizer
    tokens = tokenizer.encode(text)

    chunks = []
    for i in range(0, len(tokens), chunk_size):
        chunk = tokens[i:i+chunk_size]
        chunks.append(tokenizer.decode(chunk))

    return chunks

# Extract and split PDF text
pdf_path = "/content/TNEA_Cutoff_Explanations.pdf"
extracted_text = extract_text_from_pdf(pdf_path)
text_chunks = chunk_text(extracted_text, chunk_size=1000)

print(f"PDF split into {len(text_chunks)} chunks.")


PDF split into 287 chunks.


In [57]:
college_data_chunks = {i: chunk for i, chunk in enumerate(text_chunks)}

# Save for future use
import pickle

with open("college_data_chunks.pkl", "wb") as f:
    pickle.dump(college_data_chunks, f)

print("College data saved in chunks for retrieval.")


College data saved in chunks for retrieval.


In [58]:
# from together import Together
# import pickle

# # Load the stored chunks
# with open("college_data_chunks.pkl", "rb") as f:
#     college_data_chunks = pickle.load(f)

# # Initialize API client
# client = Together(api_key="b9503247fb87502846460e23685b040c0fb0f1a4a1221aee25d32b22217453fa")

# def retrieve_relevant_chunk(query):
#     """Find the most relevant chunk based on the user's query."""
#     relevant_chunks = []
#     for chunk in college_data_chunks.values():
#         if query.lower() in chunk.lower():
#             relevant_chunks.append(chunk)

#     return " ".join(relevant_chunks[:3])  # Limit to 3 chunks

# def query_college_cutoff(cutoff, department="COMPUTER SCIENCE AND ENGINEERING", community="BC"):
#     """Generate a dynamic query and retrieve relevant chunks."""
#     query = f"Find colleges for department {department} and community {community} with a minimum cutoff of {cutoff}."
#     relevant_data = retrieve_relevant_chunk(str(cutoff))  # Retrieve relevant data

#     prompt = f"""
#     You are an expert in Tamil Nadu Engineering Admissions (TNEA).
#     Here is the relevant college cutoff data:
#     {relevant_data}

#     Now, answer the following query:
#     {query}
#     """

#     response = client.chat.completions.create(
#         model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
#         messages=[{"role": "user", "content": prompt}],
#     )

#     return response.choices[0].message.content

# # Example query
# user_cutoff = 199.5
# result = query_college_cutoff(user_cutoff)
# print(result)


Based on the provided cutoff data, for the department "COMPUTER SCIENCE AND ENGINEERING" and community "BC" with a minimum cutoff of 199.5, there is no college that meets this criteria.

The college with code 1, "University Departments of Anna University Chennai - CEG Campus", has a minimum cutoff of 200 for BC community, which is higher than the specified minimum cutoff of 199.5.

The college with code 4, "University Departments of Anna University Chennai - MIT Campus", has a minimum cutoff of 198.5 for BC community, which is lower than the specified minimum cutoff of 199.5.

Therefore, there are no colleges that meet the specified criteria.


In [62]:
import json
from together import Together
import pickle

# Load stored chunks
with open("college_data_chunks.pkl", "rb") as f:
    college_data_chunks = pickle.load(f)

# Initialize API client
client = Together(api_key="b9503247fb87502846460e23685b040c0fb0f1a4a1221aee25d32b22217453fa")

def retrieve_relevant_chunk(query):
    """Find the most relevant chunk based on the user's query."""
    relevant_chunks = []
    for chunk in college_data_chunks.values():
        if query.lower() in chunk.lower():
            relevant_chunks.append(chunk)

    return " ".join(relevant_chunks[:3])  # Limit to 3 chunks

def query_college_cutoff(cutoff, department="COMPUTER SCIENCE AND ENGINEERING", community="BC"):
    """Generate a dynamic query and retrieve relevant chunks."""
    query = f"Find colleges for department {department} and community {community} with a minimum cutoff of {cutoff}."
    relevant_data = retrieve_relevant_chunk(str(cutoff))  # Retrieve relevant data

    prompt = f"""
    You are an expert in Tamil Nadu Engineering Admissions (TNEA).
    Here is the relevant college cutoff data:
    {relevant_data}

    Now, extract and return only the college details in JSON format:
    [{{
      "college_code": <College Code>,
      "college_name": "<College Name>"
    }}]
    """

    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt}],
    )

    # Check if response content is valid JSON before parsing
    try:
        json_result = json.loads(response.choices[0].message.content)
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON response: {response.choices[0].message.content}")
        # Handle the error, e.g., return an empty list or a default value
        return []  # or {}

    return json.dumps(json_result, indent=2)  # Pretty print JSON

# Example query
user_cutoff = 199.5
result = query_college_cutoff(user_cutoff)
print(result)

Error: Invalid JSON response: Here are the college details extracted from the given data in JSON format:

```
[
  {
    "college_code": 1,
    "college_name": "University Departments of Anna University Chennai - CEG Campus Sardar Patel Road Guindy Chennai 600 025"
  },
  {
    "college_code": 4,
    "college_name": "University Departments of Anna University Chennai - MIT Campus Chrompet Tambaram Taluk Chengalpattu District 600 044"
  }
]
```
[]
